In [1]:
# Bert test

# from transformers import DistilBertTokenizer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

# import tensorflow as tf
import pandas as pd
from DataPrep import preprocess, remove_rare_words
from sklearn.model_selection import train_test_split

from datasets import Dataset

e:\OneDrive\Study\CS Master\Natural Language Processing\Project\nlpprojectvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hein\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hein\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Logistic Regression test

genres = ['rap', 'rock', 'rb', 'country']
limit = 100

# Prepare data
df = pd.read_csv(f'data/song_lyrics_reduced_{"_".join(genres)}_{limit}.csv')

# documents = preprocess(df['lyrics'], debug=False)

In [ ]:
# documents = preprocess(df['lyrics'], debug=False)
X_train, X_test = train_test_split(df, test_size=0.1, random_state = 42)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Convert datasets to tokenized format
train_dataset = Dataset.from_pandas(X_train)
test_dataset = Dataset.from_pandas(X_test)

def tokenize_data(examples):
    return tokenizer(examples["lyrics"], truncation=True)

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map: 100%|██████████| 40/40 [00:00<00:00, 1514.83 examples/s]


In [8]:
tokenized_train

Dataset({
    features: ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'title', 'tag', 'lyrics', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 360
})

In [6]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# X_train_new = [tokenizer(x, truncation=True, padding=True, max_length=512) for x in X_train]
# X_test_new = [tokenizer(x, truncation=True, padding=True, max_length=512) for x in X_test]

In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

trainer.train()

C:\Users\Hein\AppData\Local\Temp\ipykernel_16516\2717268583.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: No columns in the dataset match the model's forward method signature: (input_ids, attention_mask, head_mask, inputs_embeds, labels, output_attentions, output_hidden_states, return_dict, labels, label_ids, label). The following columns have been ignored: [tag, title, Unnamed: 0.1, Unnamed: 0.2, lyrics, Unnamed: 0, __index_level_0__]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [6]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [7]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)

In [9]:
unique_labels = list(df['tag'].unique())

In [11]:
train_labels = []
for genre in y_train:
    train_labels.append(unique_labels.index(genre))

In [12]:
sum(train_labels)

5414

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

In [14]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [15]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


e:\OneDrive\Study\CS Master\Natural Language Processing\Project\nlpprojectvenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hein\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
TensorFlow and JAX classes are deprecated and will be removed in Tra

TypeError: 'builtins.safe_open' object is not iterable